# Tutorial 1: MDITRE workflow on 16s-based data for Bokulich diet classification
In this tutorial we demonstrate how to use MDITRE on 16S rRNA amplicon sequencing data, using the dataset from Bokulich et al., (2016). The study from Bokulich et al., (2016) investigates the dynamics of the gut microbiomes of 35 infants sampled over the first two years of life. We here explore the classifications tasks of diet breastfed versus formula-fed children

## Upstream bioinformatic analysis to generate MDITRE inputs
For 16S rRNA data MDITRE readily takes as input, (1) the output that has been generated after running [Dada2](https://benjjneb.github.io/dada2/tutorial.html) on the 16s rRNA sequences to obtain the amplicon sequencing variants (ASV) count table, and (2) the output file after running [pplacer](https://matsen.fhcrc.org/pplacer/) in order to perform phylogenetic placement of the ASV representative sequecences inferred by Dada2 on a reference 16S rRNA tree.

#### Example of running Dada2 on the Bokulich et al data
For ease of use, we provide [here] an R script that we used to run Dada2 on the raw 16S rRNA sequencing data from Bokulich et al., (2016). The demultiplexed fastq files need to be downloaded from the ENA under accession number [PRKEB14529](https://www.ebi.ac.uk/ena/browser/view/PRJEB14529?show=reads) and placed into a subfolder called raw_data. The scripts when succesfully executed returns three files which include the ASV abundance table (bokulich_filtered_subset.csv), the corresponding taxonomy table (Tax_tab.csv) and a FASTA file containing the ASV representative sequences (Seq_Var.fasta). This latter file is used as input to pplacer.

#### Example of running pplacer on Bokulich et al data
Also for ease of use, we provide [here](./pplacer_files/run_pplacer.sh) a script to run pplacer on the Dada2-determined ASV representative sequences (Seq_Var.fasta). The pplacer returns a placements.jplace file which is one of the inputs needed by MDITRE (see below). 

#### NOTE:
The Dada2 abundance file and the ```.jplace``` file from pplacer will need to be copied or moved along with the user generated sample metadata and subject metadata files (see below) in a input data folder to which point to in the MDITRE configuration file (see below).


## Loading and preprocessing the data
The following files are required by MDITRE for the data loading step. For other optional files that are accepted by the software for data loading and optional preprocessing features, please refer to our user manual [here](https://github.com/gerberlab/mditre/blob/master/mditre/docs/config_doc.pdf).

### Required files
#### Abundance data
A CSV file containing the microbial abundances generated by running Dada2. See [here](datasets/raw/bokulich/abundance.csv) for the abundance data file for this dataset.

#### Sample metadata
A CSV file that specifies an associated subject ID and timepoint for each sample ID. See [here](datasets/raw/bokulich/sample_metadata_no_repeats.csv) for the sample metadata file for this dataset.

#### Subject metadata
A CSV file that gives information about each subject, (including the value of whatever variable will be used as the host outcome for prediction (e.g., Plant-diet or Animal-diet in the David et al). See [here](datasets/raw/bokulich/subject_data.csv) for the subject metadata file for this dataset.

#### Pplacer output
A ```.jplace``` file created by placing the representative sequences from each OTU on a reference tree of known 16S rRNA amplicon sequences using the [pplacer](https://matsen.fhcrc.org/pplacer/) package. See [here](./datasets/raw/bokulich/placement.jplace) for the pplacer output file for this dataset. This will be used to create a tree of phylogenetic placements of the OTUs which is required by the model.

### Preparing a config file
The software requires user-defined config file with the desired data loading and preprocessing options. A pre-defined config file for this dataset is found [here](datasets/raw/bokulich/bdiet_reference.cfg). Looking into this file, one can see a section ```[data]``` that contains the flags specifying the locations of the required data files and also a ```[preprocessing]``` section with the optional data preprocessing flags, described in detail in our user manual. The following are the flags the user will always need to specify in the config file.

- <strong>tag</strong>: A short string used to generate the filename of the processed dataset (written to the current directory) in the following format ```<tag>_dataset_object.pickle```, for example ```bokulich_diet```
- <strong>abundance_data</strong>: Location of the abundance data file, for example, ```datasets/raw/bokulich/abundance.csv```
- <strong>sample_metadata</strong>: Location of sample metadata file, for example, ```datasets/raw/bokulich/sample_metadata_no_repeats.csv```
- <strong>subject_data</strong>: Location of subject metadata file, for example, ```datasets/raw/bokulich/subject_data.csv```
- <strong>jplace_file</strong>: Location of the pplacer generated .jplace file, for example, ```datasets/raw/bokulich/placement.jplace```
- <strong>outcome_variable</strong>: A string specifying which of the columns in the subject metadata table encodes the host status variable for prediction (For example, 'diet').
- <strong>outcome_positive_value</strong>: The value of the host status variable that corresponds to a 'positive' outcome (for example, 'fd'; which corresponds to formula diet; the choice doesn't affect the model and is just used for labeling outputs)

### Running the data loading/preprocessing utility
After defining the config file, we run the 'preprocess' utility, which produces a serialized python pickle object containing the pre-processed data, in this case ```bokulich_diet_dataset_object.pickle``` in the current folder.

In [1]:
# Load the required packages
# Make sure to have mditre package installed via pip prior to this step!
from mditre.data_utils import preprocess, ConfigParser, pickle

In [2]:
# Define the location of the config file
filename_16s_data_cfg = './datasets/raw/bokulich/bdiet_reference.cfg'

In [3]:
# Load the config file using ConfigParser package
config_16s = ConfigParser.ConfigParser()
config_16s.read(filename_16s_data_cfg)

['./datasets/raw/bokulich/bdiet_reference.cfg']

In [4]:
# Load and preprocess 16s data
dataset_16s = preprocess(config_16s)

Data imported (before any filtering:)
4780 variables, 43 subjects, 762 total samples
After filtering RSVs/OTUs with too few counts:
2304 variables, 43 subjects, 762 total samples
After filtering samples with too few counts:
2304 variables, 43 subjects, 757 total samples
After trimming dataset to specified experimental time window:
2304 variables, 43 subjects, 534 total samples
0.000000 62.500000 6
31.250000 93.750000 6
62.500000 125.000000 2
93.750000 156.250000 2
125.000000 187.500000 2
156.250000 218.750000 2
187.500000 250.000000 2
218.750000 281.250000 2
250.000000 312.500000 4
281.250000 343.750000 4
312.500000 375.000000 3
passing 0:
[  0.  36.  42.  49.  57.  61.  92. 122. 153. 183. 214. 244. 273. 294.
 300. 307. 337. 367. 375.]
0.000000 62.500000 4
31.250000 93.750000 4
62.500000 125.000000 2
93.750000 156.250000 1
125.000000 187.500000 1
156.250000 218.750000 2
187.500000 250.000000 2
218.750000 281.250000 1
250.000000 312.500000 1
281.250000 343.750000 3
312.500000 375.000000

Identifying best placements and calculating subtree weights...
Attaching sequences/OTUs to tree...
Pruning (this may take a moment)...
Calculating weights...
Aggregating data...
Finalizing aggregated data...
After phylogenetic aggregation:
3379 variables, 35 subjects, 471 total samples
Parsing taxonomic annotations.
Appying temporal filtering
Removing surplus internal nodes...
old/new nodes:
{'13199', '15391', '12594', '12618', 'sq35;size=126951;', '7995', 'sq8;size=671317;', '14883', '4070', '15392', '13107', '13205', '14543', '6257', '8926', '12663', 'sq39;size=116038;', 'sq81;size=55093;', 'sq66;size=73380;', '11796', '4420', '6270', 'sq95;size=40542;', '4250', '4078', '4263', 'sq47;size=94425;', '13282', '14541', '1430', '13223', '6168', '8224', '6411', '13110', '13105', '8249', '15410', '13078', '12562', '4175', '8818', 'sq12;size=487553;', '12602', '11795', '13192', '8002', '14437', '8970', '8051', '11826', 'sq310;size=5925;', '13164', '8219', '14670', '13149', '7850', '4057', '1

## Running the model on preprocessed data
We are now ready to run the MDITRE model for analysis, which takes the pickle file with the preprocessing data from the previous step. Running the MDITRE model will output the following information.
- The predictive ability of the model using F1 score as the quantitative measure
- A serialized pickle object containing all the information needed to interpret and visualize the learned rules.

### Quick start with default options
We recommend running MDITRE using cross-validation to assess the generalization capability of the model. Since the pytorch multiprocessing module does not support running from a Jupyter notebook, we included a python script [here ](./model_run_tutorial.py) to run the model on the prepared dataset from the previous step using default configuration options on the command line. We provide the following command that executes the script and runs a 5-fold cross-validation procedure using multiprocessing (1 process per fold) and returns logs pertaining to some training diagnostics (useful for debugging) and finally the F1 score.

```python -m torch.distributed.launch --nproc_per_node=5 model_run_tutorial.py --data ./datasets/bokulich_diet_agg_filtered.pickle --data_name bokulich_diet_test --is_16s```

The MDITRE software is optimized to run on a GPU. However, it can also run on a CPU. On a CPU, it should still take only a few minutes to run this example in multiprocessing mode. If the user has limited CPU cores, they can set the "nproc_per_node" to a lower number. The full description of the command line arguments accepted by the model code is located in the documentation [here](https://github.com/gerberlab/mditre/blob/master/mditre/docs/config_doc.pdf). A brief summary of the required options to be passed as arguments to the command are as follows.

- <strong>nproc_per_node</strong>: Number of parallel processes to be used for cross-validation procedure
- <strong>data</strong>: Location of the preprocessed data pickle file from the previous step
- <strong>data_name</strong>: A string used to create output file directory
- <strong>is_16s</strong>: Required if using 16S rRNA amplicon based dataset (omit if using metagenomics based data)

After executing the above command successfully, you can find the final F1 score from the cross-validation and also the saved pickle file containing the learned rule information ```bokulich_diet_rules_dump.pickle```, which will be used in the following rule visualization tutorial.

## Rule Visualization via GUI Tutorial
Now we demonstrate the rule visualization GUI (Graphical User Interface) capabilities of the MDITRE software. After successfully running the model code on a given dataset, the user can interactively explore the learned rules via the GUI. We show an example walkthrough of the GUI on the learned rules obtained from running the MDITRE model in the previous step.

In [5]:
# Import packages
from mditre.rule_viz import *
# Make the notebook pop up a separate matplotlib window (instead of displaying inline)
%matplotlib qt

### Specify the location of rule output pickle file output by the model code
After a successful run of the model code, the software saves a pickle file with suffix "rules_dump.pickle" in a specified location. This file contains all the necessary data to visualize the learned rules via the GUI.

In [6]:
rules_path = './bokulich_diet_rules_dump.pickle'

### Load the pickle file
The loaded file is a dictionary that will be input into the GUI code.

In [7]:
with open(rules_path, 'rb') as f:
    rule_dict = pickle.load(f)

### Invoke the RuleVisualizer utility
This opens a separate window with the rule visualization. We describe the workflow in the following steps.

In [8]:
rv = RuleVisualizer(rule_dict, './')
rv.plot_log_odds()

findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
findfont: Font family ['Arial'] not found. Falling back to DejaVu Sans.
findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.


## Rule Visualizer workflow
- After running the rule visualizer utlity as shown before, a separate window pops up showing the log-odds of all the learned rules as heatmaps. In this example, we see that the model was able to learn 2 rules (rule 1 and rule 2). The topmost panel shows the log-odds of individual rules. This is followed by 2 panels showing the predicted log-odds of each rule individually for each subject. The next panel shows the combined log-odds of both rules for each subjects. Finally, a panel showing ground truth labels for each subject (Breast-milk diet vs Formula diet). A rule with positive log-odds is predictive of the positive outcome (in this case Formula diet) and vice-versa. As we can see, both rules collectively can perfectly classify the subjects based on their outcomes. Next, in order to view the learned rules, one can click on the button (that says "Click").
- Clicking on a button to the left of the heatmap of rule 1 opens a separate window. This window shows the text of the rules on the top, followed by a panel showing the detector activations for each subject as a heatmap. As we see, the rule is predictive of Formula diet and therefore the detector is active (True) for all the subjects with Formula diet. We can also notice that the detector is active for a few of the subjects with breast-milk diet, and therefore this rule by itself cannot perfectly classify the subjects. Since this rule contains only 1 detector, we omit showing the conjunction of all the detectors in a separate panel.
- Next, the user can click on the button to the left of the detector heatmap, which opens a separate window showing the average abundances over time for the 2 groups of subjects of the selected taxa (shown as a tree on the right), along with the heatmaps of the slope within the selected time window (day 118 - 181). <strong>Note, if the figure does not fit the screen, please click on "Configure subplots" button (located to the left of the zoom tool on the top left), and then click "tight layout" option.</strong> The purpose of this visualization is to allow the user to explore the differences in the temporal dynamics of the selected taxa both within and outside the selected time window between the subjects. One can see that the abundances of subjects with Formula diet are higher (colored towards red) than the ones with Breast-milk diet (colored towrads blue) during days 118 - 181. After day 181, one can observe the abundances of both classes of subjects become more similar, which may reflect similar diets post liquid-foods in both groups.
- We also provide a button located in the bottom right, which when clicked opens a separate window with the full phylogenetic tree of taxa. Using the zoom tool located at the top left of the new window, one can see the selected taxa (with red branches) in the context of the other nearby taxa in the full phylogenetic tree.

In [9]:
# Close the GUI
plt.close("all")